In [ ]:
#!pip install xlrd==1.2.0 
!pip install --upgrade xlrd

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

список используемых ресурсов(для доработки проекта)
очистка датасета
https://proglib.io/p/moem-dataset-rukovodstvo-po-ochistke-dannyh-v-python-2020-03-27
scatter-matrix
https://www.kaggle.com/vijayprayagala/correlation-heat-map-and-scatter-matrix
регрессия
https://towardsdatascience.com/simple-and-multiple-linear-regression-in-python-c928425168f9



In [ ]:
# место для кода
xl = pd.ExcelFile('data1.xls')
xl.sheet_names
df = xl.parse("data2018")
df=df.replace(' ', np.nan)
#df[0:217]
df=df.dropna(how='all')
df

In [ ]:
df.shape

### 2
Выведите описание прочтенных данных. 


In [ ]:
# примитивные статистики
df.describe()

### 2 



In [ ]:
# размерности
print(df.shape)

In [ ]:
print(df.dtypes)

In [ ]:
# отбор числовых столбцов
df_numeric = df.select_dtypes(include=[np.number])
numeric_cols = df_numeric.columns.values
numeric_cols

In [ ]:
# отбор нечисловых столбцов
df_non_numeric = df.select_dtypes(exclude=[np.number])
non_numeric_cols = df_non_numeric.columns.values
print(non_numeric_cols)

In [ ]:
cols = df.columns[:13] # первые 13 колонок
# определяем цвета 
# желтый - пропущенные данные, синий - не пропущенные
colours = ['#000099', '#ffff00'] 
sns.heatmap(df[cols].isnull(), cmap=sns.color_palette(colours))

In [ ]:
#список долей отсутствующих записей для каждого признака
for col in df.columns:
    pct_missing = np.mean(df[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

In [ ]:
# создаем индикатор для признаков с пропущенными данными
for col in df.columns:
    missing = df[col].isnull()
    num_missing = np.sum(missing)
    
    if num_missing > 0:  
        print('created missing indicator for: {}'.format(col))
        df['{}_ismissing'.format(col)] = missing

# на основе индикатора строим гистограмму
ismissing_cols = [col for col in df.columns if 'ismissing' in col]
df['num_missing'] = df[ismissing_cols].sum(axis=1)

df['num_missing'].value_counts().reset_index().sort_values(by='index').plot.bar(x='index', y='num_missing')


In [ ]:
df

In [ ]:
df.boxplot(column=['FDI'])

In [ ]:
df.boxplot(column=['GDP_PPP'])

In [ ]:
df.boxplot(column=['GDP_GROWTH'])

In [ ]:
df.boxplot(column=['IMPORTS'])

In [ ]:
df.boxplot(column=['EXPORTS'])

In [ ]:
df.boxplot(column=['CHARGES'])

In [ ]:
df.boxplot(column=['LABOR_FORCE'])

In [ ]:
df.boxplot(column=['EDB'])

In [ ]:
df.boxplot(column=['RESERVES'])

In [ ]:
df.boxplot(column=['RATE'])

In [ ]:
df.boxplot(column=['HCI'])

In [ ]:
from pandas.plotting import scatter_matrix
#scatter_matrix(df[2:3],figsize=[20,20])
attr=list(df.columns[2:10])
attr
df1=df.loc[:,attr]
df1
pd.plotting.scatter_matrix(df1,figsize=(12,12))

In [ ]:
# описательные статистики
df.describe()

In [ ]:
#Корреляция между переменными
import matplotlib.pyplot as plt
def plot_corr_matrix(data_basic,attr,fig_no, method):
    correlations=data_basic.corr(method=method)
    fig=plt.figure(fig_no)
    ax=fig.add_subplot(111)
    ax.set_title("Correlation Matrix for Specified Attributes")
    ax.set_xticklabels(['']+attr)
    ax.set_yticklabels(['']+attr)
    cax=ax.matshow(correlations,vmax=1,vmin=-1)
    fig.colorbar(cax)
    plt.show()

In [ ]:
plot_corr_matrix(df1,attr,2, 'pearson')
plot_corr_matrix(df1,attr,2, 'kendall')
plot_corr_matrix(df1,attr,2, 'spearman')

#correlations=df1.corr(method='pearson')
#correlations

In [ ]:
import statsmodels.api as sm

In [ ]:
df=df.fillna(df.mean())
feature_names={'GDP_PPP'}
target_name={'FDI'}

In [ ]:
df=df.fillna(df.mean())
feature_names={'GDP_PPP', 'GDP_GROWTH', 'IMPORTS', 'EXPORTS', 'CHARGES',
       'LABOR_FORCE', 'EDB', 'RESERVES', 'RATE', 'HCI'}

target_name={'FDI'}

In [ ]:
X = df[feature_names]
y = df[target_name]

# Note the difference in argument order
model = sm.OLS(y, X).fit()
predictions = model.predict(X) # make the predictions by the model

# Print out the statistics
model.summary()

In [ ]:
from sklearn.linear_model import Ridge, Lasso

In [ ]:
X = df[feature_names]
y = df[target_name]
print(X, y)

In [ ]:
#split data

In [ ]:
model = Lasso(alpha=0.1, random_state=241)
model.fit(X, y)

In [ ]:
model.coef_

In [ ]:
y_pred = model.predict(X)
#print(y_pred)

In [ ]:
from matplotlib import pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import plotly

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=X.index,y = y_pred.squeeze(),mode='lines',name='y_hat'))
fig.add_trace(go.Scatter(x=X.index,y = y.squeeze(),mode='lines',name='y_fact'))
#fig.update_layout(title=f"MAPE: {mapes[-1]}%")
#figures.append(fig)
fig.show()


In [ ]:
model1 = Ridge(alpha=0.1, random_state=241)
model1.fit(X, y)
y_pred = model1.predict(X)

In [ ]:
model.coef_

In [ ]:
#0.001
model1.coef_

In [ ]:
#1000
model1.coef_

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=X.index,y = y_pred.squeeze(),mode='lines',name='y_hat'))
fig.add_trace(go.Scatter(x=X.index,y = y.squeeze(),mode='lines',name='y_fact'))
#fig.update_layout(title=f"MAPE: {mapes[-1]}%")
#figures.append(fig)
fig.show()

In [ ]:
print("Ridge coefs: ", model1.coef_)
print("Lasso coefs: ", model.coef_)

In [ ]:
print(type(np.array(y.values)))
print(type(y_pred))
np_pred = np.array(y.values).reshape(-1)
print(np_pred.shape)
print(y_pred.shape)
print(np.diff(y_pred - np.array(y.values)).shape)
print(np.diff(y_pred - np.array(y.values)))

In [ ]:
import math
def regression_model(name, X, y, alpha):
  if name == 'Ridge':
    model = Ridge(alpha=alpha, random_state=241)
  else:
    model = Lasso(alpha=alpha, random_state=241)
  model.fit(X, y)
  y_pred = model.predict(X)

  fig = go.Figure()
  fig.add_trace(go.Scatter(x=X.index,y = y_pred.squeeze(),mode='lines',name='y_hat'))
  fig.add_trace(go.Scatter(x=X.index,y = y.squeeze(),mode='lines',name='y_fact')) 
  error = 0
  for i in range(len(y_pred)):
    error += (y_pred[i] - y.iloc[i])*(y_pred[i] - y.iloc[i])
  return model, y_pred, fig, math.sqrt(error)/ len(y_pred)


In [ ]:
for alpha in [0.1, 1, 10, 100, 1000]:
  model, y_pred, fig, err = regression_model('Ridge', X, y, alpha)
  fig.update_layout(title=f"ERR: {err}")
  fig.show()
  print(err)



In [ ]:
print(X.columns)
print(list(X.median()))
print(y.median())
for alpha in [0.1, 1, 10, 100, 1000]:
  model, y_pred, fig = regression_model('Ridge', X, y, alpha)
  
  #print(model.coef_)
  print([model.coef_[0][i]*list(X.median())[i] for i in range(9)])
  

In [ ]:
for alpha in [0.1, 1, 10, 100, 1000]:
  model, y_pred, fig = regression_model('Lasso', X, y, alpha)
  
  print(model.coef_)
  #print([model.coef_[i]*list(X.median())[i] for i in range(9)])